Declare some basic symbols:
A_v denotes a variable A that is a vector
Capital letter denote Group points
Small letters denote group scalars

polynomial terms are also used in `X`, `T`, but they would be clear from context

In [364]:
from sympy import *
v, G = MatrixSymbol('v', 1, 1), MatrixSymbol('G', 1, 1)
h_n = 6
g_n = 4
gamma_v = MatrixSymbol('gamma_v', 1, h_n)
G_v = MatrixSymbol('G_v', g_n, 1)
H_v = MatrixSymbol('H_v', h_n, 1)
# T value commitment
# gamma_v scalar is multiplies to H_0, the first term of H_v. Others are 0
V = v*G + gamma_v*H_v
print(V)

gamma_v*H_v + v*G


Define
    - d_v(vector of digits of v),
    - m_v(vector of multiplicities of digits of v)
    - r_v(vector of reciprocals of v)
    - s(vector of blinding factors)

We will first attempt at dealing with the last verification equation in terms of T

$r_i = \frac{1}{(e + d_i)} => r_i \times d_i = 1 - e \times r_i$

In [510]:
from sympy.solvers.solveset import linear_coeffs
d_v, m_v, r_v, s_v = MatrixSymbol('d_v', 1, g_n), MatrixSymbol('m_v', 1, g_n), MatrixSymbol('r_v', 1, g_n), MatrixSymbol('s_v', 1, g_n)
ld_v, lm_v, lr_v, ls_v = MatrixSymbol('ld_v', 1, h_n), MatrixSymbol('lm_v', 1, h_n), MatrixSymbol('lr_v', 1, h_n), MatrixSymbol('ls_v', 1, h_n)

b_s, b_d, b_m, b_r = MatrixSymbol('b_s', 1, 1), MatrixSymbol('b_d', 1, 1), MatrixSymbol('b_m', 1, 1), MatrixSymbol('b_r', 1, 1)

D = b_d*G + d_v*G_v + ld_v*H_v
M = b_m*G + m_v*G_v + lm_v*H_v
R = b_r*G + r_v*G_v + lr_v*H_v
S = b_s*G + s_v*G_v + ls_v*H_v

# create w_v
T = symbols('T', commutative=True)
Y = symbols('Y', commutative=True)
Q = symbols('Q', commutative=True)
E = symbols('E', commutative=True)

b = symbols('b', commutative=True)
alpha_d = transpose(Matrix([b, b**2, b**3, b**4]))
alpha_m = transpose(Matrix([1/E, 1/(E + 1), 1/(E + 2), 1/(E + 3)]))

# Create the public terms
alpha_r2 = transpose(Matrix([E, E, E, E]))
one_v = transpose(Matrix([1, 1, 1, 1]))
alpha_r = -1*one_v

Q_v = transpose(Matrix([Q, Q**2, Q**3, Q**4]))
Q_inv_v = transpose(Matrix([Q**-1, Q**-2, Q**-3, Q**-4]))

# Public value
P = hadamard_product(Q_inv_v, Y*T**4*alpha_m + Y*T**2*alpha_r + T**3*alpha_d)*G_v
P = P + T**2*alpha_r2*G_v

# Consider the expression
C = S + T*M + T**2*D + T**3*R + 2*T**5*V + P
C = expand(C)
w_v = linear_coeffs(C, G_v)[0]
l_v = linear_coeffs(C, H_v)[0]
v_hat = linear_coeffs(C, G)[0]

v_hat = v_hat[0] + 2*T**5*one_v.dot(Q_v) # Reciprocal constraint constant
v_hat = v_hat + 2*T**5*((alpha_d*transpose(alpha_r2))[0, :].as_explicit()[0]) # Sum value constraint constant
v_hat = v_hat + 2*T**5*Y*((alpha_r*transpose(hadamard_product(Q_inv_v, alpha_d)))[0, :].as_explicit()[0]) # Range check constraint constant

print("Coeffs of G_v:", w_v)
print("Coeffs of H_v:", l_v)
print("Coeffs of G:", v_hat)

Coeffs of G_v: T**3*r_v + T**2*d_v + T*m_v + Matrix([[E*T**2 + T**3*b/Q - T**2*Y/Q + T**4*Y/(E*Q), E*T**2 + T**4*Y/(E*Q**2 + Q**2) + T**3*b**2/Q**2 - T**2*Y/Q**2, E*T**2 + T**4*Y/(E*Q**3 + 2*Q**3) + T**3*b**3/Q**3 - T**2*Y/Q**3, E*T**2 + T**4*Y/(E*Q**4 + 3*Q**4) + T**3*b**4/Q**4 - T**2*Y/Q**4]]) + s_v
Coeffs of H_v: T**3*lr_v + T**2*ld_v + T*lm_v + (2*T**5)*gamma_v + ls_v
Coeffs of G: 2*T**5*Y*(-b/Q - b**2/Q**2 - b**3/Q**3 - b**4/Q**4) + 2*T**5*(Q**4 + Q**3 + Q**2 + Q) + 2*T**5*(E*b**4 + E*b**3 + E*b**2 + E*b) + 2*T**5*v[0, 0] + T**3*b_r[0, 0] + T**2*b_d[0, 0] + T*b_m[0, 0] + b_s[0, 0]


The challenge $q$ is used to separate the `n` reciprocal constraints and one final sum constraint. The sum constraint occurs in $q^0$ and other constraints start from $q^1$


Norm argument:
$|\vec{w}|^2_q + \langle \vec{l},\vec{c} \rangle = v$ for a given $C = vG + \langle w, \vec{G} \rangle + \langle l,\vec{H} \rangle$

In [511]:
import sympy.vector
R = symbols('R', commutative=True)
c_v = Matrix([[T], [T**2], [T**3], [T**4], [T**6], [T**7]])
c_v = c_v*R
w_norm = hadamard_product(Q_v,w_v)*transpose(w_v)
expanded_expr = expand(w_norm[0] + (l_v*c_v)[0, :].as_explicit()[0] - v_hat)

print("Value Total Check:", expanded_expr.coeff(T, 5).coeff(R, 0).coeff(Y, 0).coeff(Q, 0))
print("Reciprocal Check:", expanded_expr.coeff(T, 5).coeff(R, 0).coeff(Y, 0).coeff(Q, 1))
print("Range Check:", expanded_expr.coeff(T, 5).coeff(R, 0).coeff(Y, 1))
print("Extra terms:", expanded_expr.coeff(T, 5).coeff(R, 1))


Value Total Check: 2*b**4*d_v[0, 3] + 2*b**3*d_v[0, 2] + 2*b**2*d_v[0, 1] + 2*b*d_v[0, 0] - 2*v[0, 0]
Reciprocal Check: 2*E*r_v[0, 0] + 2*d_v[0, 0]*r_v[0, 0] - 2
Range Check: 2*Q**4*m_v[0, 3]/(E*Q**4 + 3*Q**4) + 2*Q**3*m_v[0, 2]/(E*Q**3 + 2*Q**3) + 2*Q**2*m_v[0, 1]/(E*Q**2 + Q**2) - 2*r_v[0, 0] - 2*r_v[0, 1] - 2*r_v[0, 2] - 2*r_v[0, 3] + 2*m_v[0, 0]/E
Extra terms: ld_v[0, 2] + lm_v[0, 3] + lr_v[0, 1]


Now, let's move onto trying to adjust other terms in different powers of t that we cannot balance. Makes sense to try from last power as terms till $t^8$ are public terms. Forcing `gamma` corresponding to the terms  to be 0.

In [512]:
print("Last terms(T11):", expanded_expr.coeff(T, 11))
print("Last terms(T10):", expanded_expr.coeff(T, 10))
print("Last terms(T9):", expanded_expr.coeff(T, 9))
print("Last terms(T8):", expanded_expr.coeff(T, 8))

Last terms(T11): 2*R*gamma_v[0, 4]
Last terms(T10): R*lr_v[0, 5]
Last terms(T9): 2*R*gamma_v[0, 3] + R*ld_v[0, 5] + R*lr_v[0, 4]
Last terms(T8): Q**4*Y**2/(E**2*Q**8 + 6*E*Q**8 + 9*Q**8) + Q**3*Y**2/(E**2*Q**6 + 4*E*Q**6 + 4*Q**6) + Q**2*Y**2/(E**2*Q**4 + 2*E*Q**4 + Q**4) + 2*R*gamma_v[0, 2] + R*ld_v[0, 4] + R*lm_v[0, 5] + Y**2/(E**2*Q)


Now, we have a $t^7$ term that depends on $\vec{ls}$. It is easy for the prover to set this value so that the overall term becomes zero.

In [513]:
print("Last terms(T7):", expanded_expr.coeff(T, 7))
print("Last terms(T7):", expanded_expr.coeff(T, 7).coeff(R, 0).coeff(Q, 2))
print("Last terms(T7):", expanded_expr.coeff(T, 7).coeff(R, 1))

Last terms(T7): 2*Q**4*Y*b**4/(E*Q**8 + 3*Q**8) + 2*Q**4*Y*r_v[0, 3]/(E*Q**4 + 3*Q**4) + 2*Q**3*Y*b**3/(E*Q**6 + 2*Q**6) + 2*Q**3*Y*r_v[0, 2]/(E*Q**3 + 2*Q**3) + 2*Q**2*Y*b**2/(E*Q**4 + Q**4) + 2*Q**2*Y*r_v[0, 1]/(E*Q**2 + Q**2) + 2*R*gamma_v[0, 1] + R*lm_v[0, 4] + R*lr_v[0, 3] + R*ls_v[0, 5] + 2*Y*r_v[0, 0]/E + 2*Y*b/(E*Q)
Last terms(T7): 2*Y*b**2/(E*Q**4 + Q**4) + 2*Y*r_v[0, 1]/(E*Q**2 + Q**2)
Last terms(T7): 2*gamma_v[0, 1] + lm_v[0, 4] + lr_v[0, 3] + ls_v[0, 5]


Note: This comes back to the $t^7$ issue that we were discussing with Liam. In this setting, it is impossible to balance the $t^7$ because it depends on secret data and challenge data that is unknown yet. There are two approaches to fix this issue
- The most natural way would be to introduce a $t^7$ term in the $\vec{c}$
- The other way(as Liam suggested) would be to modify the $\vec{c}$ so that the term that generates $t^7$ from $l_r$ can be set "free". To do this, we would need to set $t^4$ power with an additional challenge.

This second route is promising and probably works, but for understanding, we focus on the first route

The logic for the remaining powers of $t^i$, $i \leq 7$ and $i \ne 5$ is the same. We have a free term from the blinded $ls$ vector that we can use to balance out the equation. 

Note that it is crucial that we cannot generate power of $t^5$ from the blinding term as it would disturb the soundness of the protocol and allow to prover to create bad proofs